# Setup

This notebook introduces the basic usage of the RigidBodySim package. Let's get started.

In [1]:
using RigidBodySim
using RigidBodyDynamics

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/d00ee8fccd1d3e94c11866a3589118e637f8d9e1-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/d00ee8fccd1d3e94c11866a3589118e637f8d9e1-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

RigidBodySim reexports select symbols from packages in the [DifferentialEquations](https://github.com/JuliaDiffEq/DifferentialEquations.jl) ecosystem. To access additional DifferentialEquations features, simply add one or more of the following:
```julia
using DiffEqBase # basics
using OrdinaryDiffEq # more ODE-related functionality
using DiffEqCallbacks # callback library
```

# Model definition

We first need to create a `RigidBodyDynamics.Mechanism` instance, representing the rigid body system we want to simulate. One way of creating a `Mechanism` is by parsing a [URDF](http://wiki.ros.org/urdf) file:

In [2]:
urdf = joinpath(dirname(pathof(RigidBodySim)), "..", "test", "urdf", "Acrobot.urdf")
mechanism = parse_urdf(Float64, urdf)
remove_fixed_tree_joints!(mechanism);

Next, we'll create a `MechanismState` object, and set the initial joint positions and velocities:

In [3]:
state = MechanismState(mechanism)
shoulder, elbow = joints(mechanism)
configuration(state, shoulder) .= 0.3
configuration(state, elbow) .= 0.4
velocity(state, shoulder) .= 1.
velocity(state, elbow) .= 2.;

# Creating an `ODEProblem`:

RigidBodySim uses RigidBodyDynamics.jl to evaluate the equations of motion, and [OrdinaryDiffEq.jl](https://github.com/JuliaDiffEq/OrdinaryDiffEq.jl) for numerical integration (see [DifferentialEquations.jl](https://github.com/JuliaDiffEq/DifferentialEquations.jl) for documentation).

RigidBodySim does not attempt to abstract away this dependence on the DifferentialEquations ecosystem, as doing so would necessarily expose only a subset of the functionality provided by DifferentialEquations, and require users familiar with the DifferentialEquations ecosystem to learn yet another API. Instead, RigidBodySim simply plugs into existing DifferentialEquations functionality, providing convenience methods and extensions.

One example of this is the `Dynamics` object, which can represent the open-loop or closed-loop dynamics of a `Mechanism`:

In [4]:
open_loop_dynamics = Dynamics(mechanism);

The `Dynamics` object is callable, and satisfies the (in-place) [function signature that the OrdinaryDiffEq package expects](http://docs.juliadiffeq.org/stable/types/ode_types.html#Problem-Type-1), i.e. `f!(du, u, p, t)`. Furthermore the arguments with which the `Dynamics` object is called may have arbitrary element types.

Using the `Dynamics` object we just created, we can now set up an `ODEProblem` as normal:

In [5]:
final_time = 1000.
problem = ODEProblem(open_loop_dynamics, state, (0., final_time))

ODEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (0.0, 1000.0)
u0: [0.3, 0.4, 1.0, 2.0]

This sets up (but doesn't solve) the initial value problem to be solved through numerical integration of the equations of motion. Note that, as a convenience, RigidBodySim provides an `ODEProblem` constructor overload that (among other things) accepts a `MechanismState` object for the initial state (`u0`) argument.

# Visualization

Next, let's set up a visualizer.

We'll first use `MechanismGeometries` to parse the link geometries for our `Mechanism` from the URDF file:

In [6]:
using MechanismGeometries
visuals = URDFVisuals(urdf);

For 3D visualization, RigidBodySim uses [MeshCat](https://github.com/rdeits/MeshCat.jl) (or more specifically, [MeshCatMechanisms](https://github.com/JuliaRobotics/MeshCatMechanisms.jl), built on top of MeshCat). A graphical user interface (GUI) instance can be created and displayed in a standalone window using

In [7]:
gui = GUI(mechanism, visuals)
if !(haskey(ENV, "CI") && Sys.KERNEL == :Linux) # don't try to open a GUI on a headless CI build
    open(gui)
end

We'll then visualize the mechanism at the chosen initial state and time:

In [8]:
vis = gui.visualizer # a MeshCatMechanisms.MechanismVisualizer
set_configuration!(vis, configuration(state));

Alternatively, we can also open the controls and the 3D visualizer in separate windows:

In [9]:
using Blink: Window
if !(haskey(ENV, "CI") && Sys.KERNEL == :Linux) # don't try to open a GUI on a headless CI build
    open(gui.controls, Window())
    open(gui.visualizer, Window());
end

# Simulating while visualizing

We don't just want to visualize the initial state; we want to visualize the mechanism during the simulation as well. In addition, we want to be able to interact with the simulation process (for example, by terminating it using the 'stop' button in the GUI).

This functionality is implemented using a set of DifferentialEquations integrator callbacks (i.e., a `DiffEqBase.CallbackSet`). RigidBodySim provides a convenience `CallbackSet` constructor for these visualizer hooks:

In [10]:
gui_callback = CallbackSet(gui);

We can now start simulating by simply calling the `DiffEqBase.solve` function, making sure to pass in our visualizer callback set as the `callback` keyword argument.

**Note**: just in time compilation may make it seem like nothing is happening for a few seconds.

In [11]:
sol = solve(problem, Tsit5(), abs_tol = 1e-10, dt = 0.05, callback = gui_callback);

**Note**: a headless simulation can of course be performed by simply omitting the `callback` keyword argument. This may be faster than visualizing while simulating.

By default, the simulation will be run as fast as possible. If this is not desired, a `RealtimeRateLimiter` callback can be used:

In [12]:
problem = ODEProblem(open_loop_dynamics, state, (0., 5.))
rate_limiter = RealtimeRateLimiter(max_rate = 1.)
callbacks = CallbackSet(gui_callback, rate_limiter) # this is how you combine callbacks
solve(problem, Tsit5(), abs_tol = 1e-10, dt = 0.05, callback = callbacks);

# Visualization after the simulation

We can also play back the simulation results after the simulation is completed. RigidBodySim provides an overload for `RigidBodyTreeInspector.animate` that takes a `DiffEqBase.ODESolution`, returned by the `solve` function. Note that this `animate` function can be used even if the original simulation was performed headless (without visualization).

Here's an example 5-second headless simulation, followed by playback at half speed:

In [13]:
problem = ODEProblem(open_loop_dynamics, state, (0., 5.))
sol = solve(problem, Vern7(), abs_tol = 1e-10, dt = 0.05);
setanimation!(vis, sol, realtime_rate=0.5);

**Note**: simulation time is currently not updated while animating, and the 'stop' and 'pause' buttons have no effect.

# Simulation with a controller in the loop

So far, we've only simulated a passive mechanism. We can also simulate with a controller in the loop.

## Basics

The basic approach to controller-in-the-loop simulation is to pass in an a function of the form
```julia
function (τ::AbstractVector, t, state::MechanismState)
    # set τ based on t and state...
end
```
as an extra argument to the `ODEProblem` constructor. For example:

In [14]:
function control!(τ, t, state)
    view(τ, velocity_range(state, shoulder))  .= 5 * sin(t)
    view(τ, velocity_range(state, elbow)) .= -configuration(state, shoulder)
end

control! (generic function with 1 method)

In [15]:
closed_loop_dynamics = Dynamics(mechanism, control!)
zero!(state)
controlproblem = ODEProblem(closed_loop_dynamics, state, (0., 5.))
sol = solve(controlproblem, Vern7(), abs_tol = 1e-10, dt = 0.05)
setanimation!(vis, sol);

## Simulating a digital controller

In the previous example, the function `control!` is called whenever the integrator queries the dynamics. There are cases where this is not desired. For example, a digital controller typically runs at some fixed frequency, independent of the integrator time step. For such cases, `RigidBodySim` provides the `PeriodicController` type:

In [16]:
τ = similar(velocity(state))
Δt = 1e-3 # time between calls to the `control!` function
controller = PeriodicController(τ, Δt, control!);

which can be passed into the `ODEProblem` constructor in similar fashion:

In [17]:
zero!(state)
problem = ODEProblem(Dynamics(mechanism, controller), state, (0., 5.))

ODEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (0.0, 5.0)
u0: [0.0, 0.0, 0.0, 0.0]

Under the hood, this creates an `ODEProblem` that contains a [`DiffEqCallbacks.PeriodicCallback`](http://docs.juliadiffeq.org/stable/features/callback_library.html#PeriodicCallback-1), which ensures that the `control!` function is called at the desired rate.

In [18]:
sol = solve(problem, Tsit5(), abs_tol = 1e-10, dt = 0.05, callback = gui_callback);

and animate the results:

In [19]:
setanimation!(vis, sol);